## Pre-Training [`GPT-Neo`](https://github.com/huggingface/transformers/blob/main/src/transformers/models/gpt_neo/modeling_gpt_neo.py) & [`RoBERTa`](https://github.com/huggingface/transformers/blob/main/src/transformers/models/roberta/modeling_roberta.py) on [`TinyStories`](https://huggingface.co/datasets/roneneldan/TinyStories)

This script shows how to pre-train both models. I put them in one notebook because the majority of the code is shared; but you may want to separate the logic per model. 

In [40]:
# %pip install torch wandb transformers[torch] datasets tqdm 
%load_ext autoreload
%autoreload 2 

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [41]:
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "0"  # or "0,1" for multiple GPUs
from typing import Optional, Tuple
import wandb; wandb.login()
import torch
from transformers import (
    RobertaForMaskedLM, RobertaConfig, RobertaTokenizerFast,
    GPTNeoForCausalLM, GPTNeoConfig, GPT2TokenizerFast
)
debug_mode = True

wandb: WARNING Calling wandb.login() after wandb.init() has no effect.


#### Models 
We consider GPT-Neo and BERT as base transformer architectures. This consists of the following blocks linked via residual connections:

- Embeddings: Map one-hot (sparse) token vectors to a dense vector of length `hidden_size`. Add positional encoding. 
- $n$ Blocks: Contain self-attention and FFNs.
- Head: Map hidden state back to a useful output. 

This specifies some of the model-related hyperparameters. I chose them based on what achieved reasonable performance in the [TinyStories paper](https://arxiv.org/abs/2305.07759), while also being feasible to train on our limited compute budgets. 

In [42]:
config_gpt = dict(

    # EMBEDDING PARAMETERS
    vocab_size              = 10_000,   # number of tokens in the vocabulary 
    hidden_size             = 516,      # embedding size (vector length) of each token 
    max_position_embeddings = 512,      # maximum sequence length (context window)

    # BLOCKS (ATTN & FFN)
    num_layers          = 2,                    # number of transformer blocks
    attention_types     = [[["global", "local"], 1]], # (GPT-Neo-specific) global and local attention 
    num_heads           = 4,                    # attention heads
    window_size         = 256,                  # (GPT-Neo-specific) for local attention 
    intermediate_size   = 1024,                 # size of 'up-projection' layer in FFN

    pad_token_id = 0,           # need to specify this for tokenizer interop between models
)

config_rob = dict(
    
    # EMBEDDING PARAMETERS
    vocab_size              = 10_000,   
    hidden_size             = 516,      
    # we add 1 as RoBERTa uses a special position embedding for the padding token (zero vector)
    max_position_embeddings = config_gpt['max_position_embeddings'] + 1,

    # BLOCKS (of course naming is different in roberta :) )
    num_hidden_layers = config_gpt['num_layers'],
    num_attention_heads = config_gpt['num_heads'],
    intermediate_size=1024,                     

    pad_token_id = 0,
)

config_gpt = GPTNeoConfig(**config_gpt)
config_rob = RobertaConfig(**config_rob)

In [43]:
import torch.nn.functional as F
import torch.nn as nn
import torch



In [65]:

class GPTNeoSelfAttention3(nn.Module):
    def __init__(self, config, attention_type):
        super().__init__()
        self.config = config

        max_positions = config.max_position_embeddings
        bias = torch.tril(torch.ones((max_positions, max_positions), dtype=bool)).view(
            1, 1, max_positions, max_positions
        )
        print(attention_type)
        print(bias)
        # local causal self attention is a sliding window where each token can only attend to the previous
        # window_size tokens. This is implemented by updating the causal mask such that for each token
        # all other tokens are masked except the previous window_size tokens.
        if attention_type == "local":
            bias = torch.bitwise_xor(bias, torch.tril(bias, -config.window_size))
        print(bias)
        self.register_buffer("bias", bias, persistent=False)
        self.register_buffer("masked_bias", torch.tensor(-1e9), persistent=False)

        self.attn_dropout = nn.Dropout(float(config.attention_dropout))
        self.resid_dropout = nn.Dropout(float(config.resid_dropout))
        self.is_causal = True

        self.embed_dim = config.hidden_size
        self.num_heads = config.num_heads
        self.head_dim = self.embed_dim // self.num_heads
        if self.head_dim * self.num_heads != self.embed_dim:
            raise ValueError(
                f"embed_dim must be divisible by num_heads (got `embed_dim`: {self.embed_dim} and `num_heads`:"
                f" {self.num_heads})."
            )

        self.k_proj = nn.Linear(self.embed_dim, self.embed_dim, bias=False)
        self.v_proj = nn.Linear(self.embed_dim, self.embed_dim, bias=False)
        self.q_proj = nn.Linear(self.embed_dim, self.embed_dim, bias=False)
        self.out_proj = nn.Linear(self.embed_dim, self.embed_dim, bias=True)

    def _split_heads(self, tensor, num_heads, attn_head_size):
        """
        Splits hidden_size dim into attn_head_size and num_heads
        """
        new_shape = tensor.size()[:-1] + (num_heads, attn_head_size)
        tensor = tensor.view(new_shape)
        return tensor.permute(0, 2, 1, 3)  # (batch, head, seq_length, head_features)

    def _merge_heads(self, tensor, num_heads, attn_head_size):
        """
        Merges attn_head_size dim and num_attn_heads dim into hidden_size
        """
        tensor = tensor.permute(0, 2, 1, 3).contiguous()
        new_shape = tensor.size()[:-2] + (num_heads * attn_head_size,)
        return tensor.view(new_shape)

    def _attn(self, query, key, value, attention_mask=None, head_mask=None):
        # Keep the attention weights computation in fp32 to avoid overflow issues
        query = query.to(torch.float32)
        key = key.to(torch.float32)

        attn_weights = torch.matmul(query, key.transpose(-1, -2))

        query_length, key_length = query.size(-2), key.size(-2)
        causal_mask = self.bias[:, :, key_length - query_length : key_length, :key_length]
        mask_value = torch.finfo(attn_weights.dtype).min
        # Need to be a tensor, otherwise we get error: `RuntimeError: expected scalar type float but found double`.
        # Need to be on the same device, otherwise `RuntimeError: ..., x and y to be on the same device`
        mask_value = torch.tensor(mask_value, dtype=attn_weights.dtype).to(attn_weights.device)
        attn_weights = torch.where(causal_mask, attn_weights, mask_value)
        
        if attention_mask is not None:
            # Apply the attention mask
            attn_weights = attn_weights + attention_mask

        attn_weights = nn.functional.softmax(attn_weights, dim=-1)
        attn_weights = attn_weights.to(value.dtype)
        attn_weights = self.attn_dropout(attn_weights)

        # Mask heads if we want to
        if head_mask is not None:
            attn_weights = attn_weights * head_mask

        attn_output = torch.matmul(attn_weights, value)

        return attn_output, attn_weights

    def forward(
        self,
        hidden_states,
        attention_mask=None,
        layer_past=None,
        head_mask=None,
        use_cache=False,
        output_attentions=False,
    ):
        query = self.q_proj(hidden_states)
        key = self.k_proj(hidden_states)
        value = self.v_proj(hidden_states)

        query = self._split_heads(query, self.num_heads, self.head_dim)
        key = self._split_heads(key, self.num_heads, self.head_dim)
        value = self._split_heads(value, self.num_heads, self.head_dim)

        if layer_past is not None:
            past_key = layer_past[0]
            past_value = layer_past[1]
            key = torch.cat((past_key, key), dim=-2)
            value = torch.cat((past_value, value), dim=-2)

        if use_cache is True:
            present = (key, value)
        else:
            present = None

        attn_output, attn_weights = self._attn(query, key, value, attention_mask, head_mask)

        attn_output = self._merge_heads(attn_output, self.num_heads, self.head_dim)
        attn_output = self.out_proj(attn_output)
        attn_output = self.resid_dropout(attn_output)

        outputs = (attn_output, present)
        if output_attentions:
            outputs += (attn_weights,)

        return outputs  # a, present, (attentions)
    
class CustomGPTNeoForCausalLM(GPTNeoForCausalLM):
    def __init__(self, config):
        super().__init__(config)
        # Assuming attention types are specified in config.attention_layers
        # Replace the standard attention with the specified type
        for block in self.transformer.h:
            block.attn.attention = GPTNeoSelfAttention3(config_gpt, block.attn.attention_type)



gpt = CustomGPTNeoForCausalLM(config=config_gpt)


global
tensor([[[[ True, False, False,  ..., False, False, False],
          [ True,  True, False,  ..., False, False, False],
          [ True,  True,  True,  ..., False, False, False],
          ...,
          [ True,  True,  True,  ...,  True, False, False],
          [ True,  True,  True,  ...,  True,  True, False],
          [ True,  True,  True,  ...,  True,  True,  True]]]])
tensor([[[[ True, False, False,  ..., False, False, False],
          [ True,  True, False,  ..., False, False, False],
          [ True,  True,  True,  ..., False, False, False],
          ...,
          [ True,  True,  True,  ...,  True, False, False],
          [ True,  True,  True,  ...,  True,  True, False],
          [ True,  True,  True,  ...,  True,  True,  True]]]])
local
tensor([[[[ True, False, False,  ..., False, False, False],
          [ True,  True, False,  ..., False, False, False],
          [ True,  True,  True,  ..., False, False, False],
          ...,
          [ True,  True,  True,  ...

In [45]:
# import torch.nn.functional as F
# import torch.nn as nn

# class GPTNeoSelfAttention2(nn.Module):
#     def __init__(self, config, attention_type):
#         super().__init__()
#         self.config = config

#         max_positions = config.max_position_embeddings
#         bias = torch.tril(torch.ones((max_positions, max_positions), dtype=bool)).view(
#             1, 1, max_positions, max_positions
#         )

#         # local causal self attention is a sliding window where each token can only attend to the previous
#         # window_size tokens. This is implemented by updating the causal mask such that for each token
#         # all other tokens are masked except the previous window_size tokens.
#         if attention_type == "local":
#             bias = torch.bitwise_xor(bias, torch.tril(bias, -config.window_size))

#         self.register_buffer("bias", bias, persistent=False)
#         self.register_buffer("masked_bias", torch.tensor(-1e9), persistent=False)

#         self.attn_dropout = nn.Dropout(float(config.attention_dropout))
#         self.resid_dropout = nn.Dropout(float(config.resid_dropout))
#         self.is_causal = True

#         self.embed_dim = config.hidden_size
#         self.num_heads = config.num_heads
#         self.head_dim = self.embed_dim // self.num_heads
#         if self.head_dim * self.num_heads != self.embed_dim:
#             raise ValueError(
#                 f"embed_dim must be divisible by num_heads (got `embed_dim`: {self.embed_dim} and `num_heads`:"
#                 f" {self.num_heads})."
#             )
        
#         self.k_proj = nn.Linear(self.embed_dim, self.embed_dim, bias=False)
#         self.v_proj = nn.Linear(self.embed_dim, self.embed_dim, bias=False)
#         self.q_proj = nn.Linear(self.embed_dim, self.embed_dim, bias=False)
#         self.out_proj = nn.Linear(self.embed_dim, self.embed_dim, bias=True)
#         self.beta = nn.Parameter(torch.zeros(1))
#         self.ELU = nn.ELU()
#         self.register_buffer("M", torch.zeros(self.num_heads, self.head_dim, self.head_dim))
#         self.register_buffer("z", torch.zeros(self.num_heads, self.head_dim))


#     def _split_heads(self, tensor, num_heads, attn_head_size):
#         """
#         Splits hidden_size dim into attn_head_size and num_heads
#         """
#         new_shape = tensor.size()[:-1] + (num_heads, attn_head_size)
#         tensor = tensor.view(new_shape)
#         return tensor.permute(0, 2, 1, 3)  # (batch, head, seq_length, head_features)

#     def _merge_heads(self, tensor, num_heads, attn_head_size):
#         """
#         Merges attn_head_size dim and num_attn_heads dim into hidden_size
#         """
#         tensor = tensor.permute(0, 2, 1, 3).contiguous()
#         new_shape = tensor.size()[:-2] + (num_heads * attn_head_size,)
#         return tensor.view(new_shape)

#     def _attn(self, query, key, value, compressive_memory, z, attention_mask=None, head_mask=None):
#         # Keep the attention weights computation in fp32 to avoid overflow issues
#         query = query.to(torch.float32)
#         key = key.to(torch.float32)

#         sigma_q = self.ELU(query[:, :, :, :]) + 1.0
#         sigma_k = self.ELU(key[:, :, :, :]) + 1.0

#         A_mem = ((sigma_q @ compressive_memory) / ((sigma_q @ z) + 1e-6))
#         A_dot = query[:, :, :, :] @ key[:, :, :, :].transpose(-2, -1)

#         query_length, key_length = query.size(-2), key.size(-2)
#         causal_mask = self.bias[:, :, key_length - query_length : key_length, :key_length]
#         mask_value = torch.finfo(A_dot.dtype).min
#         # Need to be a tensor, otherwise we get error: `RuntimeError: expected scalar type float but found double`.
#         # Need to be on the same device, otherwise `RuntimeError: ..., x and y to be on the same device`
#         mask_value = torch.tensor(mask_value, dtype=A_dot.dtype).to(A_dot.device)
#         A_dot = torch.where(causal_mask, A_dot, mask_value)

#         if attention_mask is not None:
#             # Apply the attention mask
#             A_dot = A_dot + attention_mask

#         # attn_weights = nn.functional.softmax(attn_weights, dim=-1)
#         A_dot = A_dot.to(value.dtype)
#         A_dot = self.attn_dropout(A_dot)

#         # Mask heads if we want to
#         if head_mask is not None:
#             A_dot = A_dot * head_mask
            
#         A_dot = nn.functional.softmax(A_dot / torch.sqrt(torch.tensor(self.head_dim)), dim = -1)




#         A_dot = torch.matmul(A_dot, value)
#         attention_output = (F.sigmoid(self.beta) * A_mem) + ((1 - F.sigmoid(self.beta)) * A_dot)

#         delta = (sigma_k @ compressive_memory) / ((sigma_k @ z) + 1e-6)
#         compressive_memory = compressive_memory + (sigma_k.transpose(-2, -1) @ (value[:, :, :, :] - delta))

#         z = z + sigma_k.sum(dim = -2, keepdim = True)

#         return attention_output, A_dot

#     def forward(
#         self,
#         hidden_states,
#         attention_mask=None,
#         layer_past=None,
#         head_mask=None,
#         use_cache=False,
#         output_attentions=False,
#     ):
#         query = self.q_proj(hidden_states)
#         key = self.k_proj(hidden_states)
#         value = self.v_proj(hidden_states)
#         device = query.device
#         query = self._split_heads(query, self.num_heads, self.head_dim)
#         key = self._split_heads(key, self.num_heads, self.head_dim)
#         value = self._split_heads(value, self.num_heads, self.head_dim)

#         compressive_memory = torch.zeros((self.num_heads, self.head_dim, self.head_dim), device=device)
#         z = torch.zeros((self.num_heads, self.head_dim, 1), device=device)

#         if layer_past is not None:
#             past_key = layer_past[0]
#             past_value = layer_past[1]
#             key = torch.cat((past_key, key), dim=-2)
#             value = torch.cat((past_value, value), dim=-2)

#         if use_cache is True:
#             present = (key, value)
#         else:
#             present = None

#         attn_output, attn_weights = self._attn(query, key, value, compressive_memory, z, attention_mask, head_mask)

#         attn_output = self._merge_heads(attn_output, self.num_heads, self.head_dim)
#         attn_output = self.out_proj(attn_output)
#         attn_output = self.resid_dropout(attn_output)

#         outputs = (attn_output, present)
#         if output_attentions:
#             outputs += (attn_weights,)

#         return outputs  # a, present, (attentions)

In [46]:

# # Copied from transformers.models.llama.modeling_llama.repeat_kv
# def repeat_kv(hidden_states: torch.Tensor, n_rep: int) -> torch.Tensor:
#     """
#     This is the equivalent of torch.repeat_interleave(x, dim=1, repeats=n_rep). The hidden states go from (batch,
#     num_key_value_heads, seqlen, head_dim) to (batch, num_attention_heads, seqlen, head_dim)
#     """
#     batch, num_key_value_heads, slen, head_dim = hidden_states.shape
#     if n_rep == 1:
#         return hidden_states
#     hidden_states = hidden_states[:, :, None, :, :].expand(batch, num_key_value_heads, n_rep, slen, head_dim)
#     return hidden_states.reshape(batch, num_key_value_heads * n_rep, slen, head_dim)


# class InfiniAttention(nn.Module):
#     def __init__(self, config, layer_idx= None):
#         super().__init__()
#         self.config = config
#         self.layer_idx = layer_idx
#         self.hidden_size = config.hidden_size
#         self.num_heads = config.num_attention_heads
#         self.head_dim = self.hidden_size // self.num_heads
#         self.num_key_value_heads = config.num_key_value_heads
#         self.num_key_value_groups = self.num_heads // self.num_key_value_heads
#         self.max_position_embeddings = config.max_position_embeddings
#         self.rope_theta = config.rope_theta
#         self.is_causal = True
#         self.attention_dropout = config.attention_dropout

#         if (self.head_dim * self.num_heads) != self.hidden_size:
#             raise ValueError(
#                 f"hidden_size must be divisible by num_heads (got `hidden_size`: {self.hidden_size}"
#                 f" and `num_heads`: {self.num_heads})."
#             )
#         self.q_proj = nn.Linear(self.hidden_size, self.num_heads * self.head_dim, bias=True)
#         self.k_proj = nn.Linear(self.hidden_size, self.num_key_value_heads * self.head_dim, bias=True)
#         self.v_proj = nn.Linear(self.hidden_size, self.num_key_value_heads * self.head_dim, bias=True)
#         self.o_proj = nn.Linear(self.num_heads * self.head_dim, self.hidden_size, bias=False)

#         self.rotary_emb = RotaryEmbedding(
#             self.head_dim,
#             max_position_embeddings=self.max_position_embeddings,
#             base=self.rope_theta,
#         )

#         self.beta = nn.Parameter(torch.randn(1))
#         self.register_buffer("M", torch.zeros(self.num_heads, self.head_dim, self.head_dim))
#         self.register_buffer("z", torch.zeros(self.num_heads, self.head_dim))
#         self.segment_size = 2048

#     def forward(
#         self,
#         hidden_states: torch.Tensor,
#         attention_mask: Optional[torch.Tensor] = None,
#         position_ids: Optional[torch.LongTensor] = None,
#         past_key_value: Optional[Cache] = None,
#         output_attentions: bool = False,
#         use_cache: bool = False,
#         **kwargs,
#     ) -> Tuple[torch.Tensor, Optional[torch.Tensor], Optional[Tuple[torch.Tensor]]]:
        

#         bsz, q_len, _ = hidden_states.size()

#         query_states = self.q_proj(hidden_states)
#         key_states = self.k_proj(hidden_states)
#         value_states = self.v_proj(hidden_states)

#         query_states = query_states.view(bsz, q_len, self.num_heads, self.head_dim).transpose(1, 2)
#         key_states = key_states.view(bsz, q_len, self.num_key_value_heads, self.head_dim).transpose(1, 2)
#         value_states = value_states.view(bsz, q_len, self.num_key_value_heads, self.head_dim).transpose(1, 2)

#         kv_seq_len = key_states.shape[-2]
#         if past_key_value is not None:
#             kv_seq_len += past_key_value.get_usable_length(kv_seq_len, self.layer_idx)
#         cos, sin = self.rotary_emb(value_states, seq_len=kv_seq_len)

#         query_states, key_states = apply_rotary_pos_emb(query_states, key_states, cos, sin, position_ids)

#         if past_key_value is not None:
#             cache_kwargs = {"sin": sin, "cos": cos}  # Specific to RoPE models
#             key_states, value_states = past_key_value.update(key_states, value_states, self.layer_idx, cache_kwargs)


#         if attention_mask is not None:
#             if attention_mask.size() != (bsz, 1, q_len, kv_seq_len):
#                 raise ValueError(
#                     f"Attention mask should be of size {(bsz, 1, q_len, kv_seq_len)}, but is {attention_mask.size()}"
#                 )


#         # GQA
#         # Memory retrieval and attention calculation per segment
#         memory_output = self._retrieve_from_memory(query_states, self.M, self.z)
#         # Update memory with current segment's key and value states
#         self.M, self.z  = self._update_memory(key_states, value_states, self.M, self.z)
#         key_states = repeat_kv(key_states, self.num_key_value_groups)
#         value_states = repeat_kv(value_states, self.num_key_value_groups)

#         causal_mask = attention_mask

#         attn_output = torch.nn.functional.scaled_dot_product_attention(
#             query_states,
#             key_states,
#             value_states,
#             attn_mask=causal_mask,
#             dropout_p=self.attention_dropout if self.training else 0.0,
#         )

#         combined_output = self._long_term_injection(attn_output, memory_output)

#         # Prepare output for this segment
#         combined_output = combined_output.transpose(1, 2).contiguous()
#         combined_output = combined_output.view(bsz, q_len, self.hidden_size)
#         final_output = self.o_proj(combined_output)
#         return final_output, None, past_key_value

#     def _retrieve_from_memory(self, Q, M, z):
#         # Retrieve context from compressive memory using linear attention (Eq. 3)
#         M_s_1 = torch.matmul(F.elu(Q) + 1, M)
#         Z_s_1 = torch.matmul(F.elu(Q) + 1, z.unsqueeze(-1)) + 1e-8
#         A_mem = M_s_1 / Z_s_1
#         return A_mem

#     def _update_memory(self, K, V, M, z, use_delta=False):
#         if use_delta:
#             V_retrieved = torch.matmul(F.elu(K) + 1, M) / (torch.matmul(F.elu(K) + 1, z.unsqueeze(-1)) + 1e-8)
#             updated_M = M + torch.matmul(F.elu(K).transpose(-2, -1) + 1, V - V_retrieved)
#         else:
#             updated_M = M + torch.matmul(F.elu(K).transpose(-2, -1) + 1, V)
        
#         updated_z = z + (F.elu(K) + 1).sum(dim=-2)
#         M = updated_M
#         z = updated_z
#         return M, z

#     def _long_term_injection(self, A_dot, A_mem):
#         beta = torch.sigmoid(self.beta)
#         A = beta * A_mem + (1 - beta) * A_dot
#         return A
#     def reset_memory(self):
#         self.M.zero_()
#         self.zero_()

# class RotaryEmbedding(nn.Module):
#     def __init__(self, dim, max_position_embeddings=2048, base=10000, device=None):
#         super().__init__()

#         self.dim = dim
#         self.max_position_embeddings = max_position_embeddings
#         self.base = base
#         inv_freq = 1.0 / (self.base ** (torch.arange(0, self.dim, 2, dtype=torch.int64).float().to(device) / self.dim))
#         self.register_buffer("inv_freq", inv_freq, persistent=False)

#         # Build here to make `torch.jit.trace` work.
#         self._set_cos_sin_cache(
#             seq_len=max_position_embeddings, device=self.inv_freq.device, dtype=torch.get_default_dtype()
#         )

#     def _set_cos_sin_cache(self, seq_len, device, dtype):
#         self.max_seq_len_cached = seq_len
#         t = torch.arange(self.max_seq_len_cached, device=device, dtype=torch.int64).type_as(self.inv_freq)

#         freqs = torch.outer(t, self.inv_freq)
#         # Different from paper, but it uses a different permutation in order to obtain the same calculation
#         emb = torch.cat((freqs, freqs), dim=-1)
#         self.register_buffer("cos_cached", emb.cos().to(dtype), persistent=False)
#         self.register_buffer("sin_cached", emb.sin().to(dtype), persistent=False)

#     def forward(self, x, seq_len=None):
#         # x: [bs, num_attention_heads, seq_len, head_size]
#         if seq_len > self.max_seq_len_cached:
#             self._set_cos_sin_cache(seq_len=seq_len, device=x.device, dtype=x.dtype)

#         return (
#             self.cos_cached[:seq_len].to(dtype=x.dtype),
#             self.sin_cached[:seq_len].to(dtype=x.dtype),
#         )


# # Copied from transformers.models.llama.modeling_llama.rotate_half
# def rotate_half(x):
#     """Rotates half the hidden dims of the input."""
#     x1 = x[..., : x.shape[-1] // 2]
#     x2 = x[..., x.shape[-1] // 2 :]
#     return torch.cat((-x2, x1), dim=-1)


# # Copied from transformers.models.mistral.modeling_mistral.apply_rotary_pos_emb
# def apply_rotary_pos_emb(q, k, cos, sin, position_ids, unsqueeze_dim=1):
#     """Applies Rotary Position Embedding to the query and key tensors.

#     Args:
#         q (`torch.Tensor`): The query tensor.
#         k (`torch.Tensor`): The key tensor.
#         cos (`torch.Tensor`): The cosine part of the rotary embedding.
#         sin (`torch.Tensor`): The sine part of the rotary embedding.
#         position_ids (`torch.Tensor`):
#             The position indices of the tokens corresponding to the query and key tensors. For example, this can be
#             used to pass offsetted position ids when working with a KV-cache.
#         unsqueeze_dim (`int`, *optional*, defaults to 1):
#             The 'unsqueeze_dim' argument specifies the dimension along which to unsqueeze cos[position_ids] and
#             sin[position_ids] so that they can be properly broadcasted to the dimensions of q and k. For example, note
#             that cos[position_ids] and sin[position_ids] have the shape [batch_size, seq_len, head_dim]. Then, if q and
#             k have the shape [batch_size, heads, seq_len, head_dim], then setting unsqueeze_dim=1 makes
#             cos[position_ids] and sin[position_ids] broadcastable to the shapes of q and k. Similarly, if q and k have
#             the shape [batch_size, seq_len, heads, head_dim], then set unsqueeze_dim=2.
#     Returns:
#         `tuple(torch.Tensor)` comprising of the query and key tensors rotated using the Rotary Position Embedding.
#     """
#     cos = cos[position_ids].unsqueeze(unsqueeze_dim)
#     sin = sin[position_ids].unsqueeze(unsqueeze_dim)
#     q_embed = (q * cos) + (rotate_half(q) * sin)
#     k_embed = (k * cos) + (rotate_half(k) * sin)
#     return q_embed, k_embed

In [47]:
# class CustomGPTNeoForCausalLM(GPTNeoForCausalLM):
#     def __init__(self, config):
#         super().__init__(config)
#         # Assuming attention types are specified in config.attention_layers
#         # Replace the standard attention with the specified type
#         for block in self.transformer.h:
#             block.attn.attention = GPTNeoSelfAttention(config_gpt, block.attn.attention_type)
# model = CustomGPTNeoForCausalLM( 
#     config = config_gpt,
# )
# print(f'The model has {model.num_parameters():,} parameters.')


In [48]:
# TODO: you should set all pytorch & huggingface seeds here as model initialisation depends on it

# gpt = CustomGPTNeoForCausalLM(config=config_gpt)
rob = RobertaForMaskedLM(config=config_rob)

print(f'''
    This GPT has {gpt.num_parameters():,} parameters,
     and ROB has {rob.num_parameters():,} parameters.
    ''')

# gpt, rob # uncomment to see model architecture

tensor([[[[ True, False, False,  ..., False, False, False],
          [ True,  True, False,  ..., False, False, False],
          [ True,  True,  True,  ..., False, False, False],
          ...,
          [ True,  True,  True,  ...,  True, False, False],
          [ True,  True,  True,  ...,  True,  True, False],
          [ True,  True,  True,  ...,  True,  True,  True]]]])
tensor([[[[ True, False, False,  ..., False, False, False],
          [ True,  True, False,  ..., False, False, False],
          [ True,  True,  True,  ..., False, False, False],
          ...,
          [False, False, False,  ...,  True, False, False],
          [False, False, False,  ...,  True,  True, False],
          [False, False, False,  ...,  True,  True,  True]]]])

    This GPT has 9,677,048 parameters,
     and ROB has 9,959,496 parameters.
    


In [49]:
from transformers import PreTrainedTokenizer, PretrainedConfig

def get_tokenizer_for_config(Tok: PreTrainedTokenizer, config: PretrainedConfig):

    tokenizer = Tok.from_pretrained(
        '10k-tok',                                         # our custom tokenizer
        model_max_length=config.max_position_embeddings    # sequence length (context window)
    )

    # we're using our special tokenizer with only 10'000 tokens instead of 50'256
    assert tokenizer.vocab_size == config.vocab_size

    print(f'padding token is {tokenizer.pad_token}')
    print(f'padding token in config: {config.pad_token_id}, in tokeniser: {tokenizer.pad_token_id}')
    
    return tokenizer 

tok_gpt = get_tokenizer_for_config(GPT2TokenizerFast, config_gpt)
tok_rob = get_tokenizer_for_config(RobertaTokenizerFast, config_rob)

The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'RobertaTokenizer'. 
The class this function is called from is 'GPT2TokenizerFast'.


padding token is <pad>
padding token in config: 0, in tokeniser: 0
padding token is <pad>
padding token in config: 0, in tokeniser: 0


In [50]:
from datasets import load_from_disk 

if debug_mode:
    tokenized_dataset = load_from_disk(f'./tokenized_dataset_small')
else:
    tokenized_dataset = load_from_disk(f'./tokenized_dataset')


train_dataset = tokenized_dataset['train']
eval_dataset  = tokenized_dataset['validation']

assert len(tokenized_dataset['train'][0]['input_ids']) == config_gpt.max_position_embeddings
tokenized_dataset['train'][0]['input_ids'][-10:]
# should be pad tokens (0), given that most short stories are <512 tokens

[0, 0, 0, 0, 0, 0, 0, 0, 0, 0]

#### Training
Before we get started, you may want to specify which GPU to use. See the first cell in this notebook; make sure to run it before anything else. 

Huggingface provides some powerful (and often confusingly long) APIs for model training. The `TrainingArguments` specifies our hyperparameters, which are used by the `Trainer` taking in the remaining objects (like `model`, `tokenizer`, and `train_dataset`). Specifically:

- `learning_rate` and `num_train_epochs` determine how much the model learns. A higher rate is faster, but more unstable. More epochs (entire passes over the dataset) yields incrementally better results, at the cost of more training time. 
- Batch sizes determine how many samples the model sees in *parallel*. Given `gradient_accumulation_steps=1` and a `batch_size=8`, the model will backpropagate the average loss of 8 samples; if `batch_size=1`, it will average the loss of `gradient_accumulation_steps` samples. It is important to make sure the backpropagated loss is averaged over the same number of samples, when comparing models. 

- `data_collator` batches (and optionally, pads) the input for the model. We have already padded in our `tokenized_dataset`, and leaving this argument empty will automatically batch the inputs. So why do we need it? 

    Glad you asked. This has to do with how the loss is computed in causal language modelling. In our case, we try to predict $p(y | x)$, where $x$ is an input sequence of tokens, and $y$ is the next token following that sequence. Our model, unaware of the target token $y$, outputs $\hat y$. 
    
    For `Trainer` to compute the (cross-entropy) loss, we need to provide it with both $y$ and $\hat y$. The `DataCollatorForLanguageModeling` knows this, and provides the next token $y$ as a separate part of the input, to the `Trainer`.

    The loss is the backbone of backpropagation, which we need to actually improve our model. If this is confusing, please re-watch Karpathy's GPT tutorial. 

If you prefer to write the training loop yourself, check out HF' `run_clm_no_trainer.py` scripts. (`run_mlm_no_trainer.py` for RoBERTa-style masked-language modelling, as opposed to causal language modelling). This can be useful to give you better control over which devices are used for training. 

In [51]:
from transformers import Trainer, TrainingArguments, DataCollatorForLanguageModeling

def get_hyperparameters(model, dataset):
    ''' common hyperparameters to give to the trainer '''

    # TRAINING HYPERPARAMETERS 
    batch_size = 16                  # TinyStories uses 80, but I am training locally on my poor M1 Air
    num_train_epochs = 1             # TinyStories doesn't mention
    gradient_accumulation_steps = 16 # TinyStories uses 16

    lr = 5e-4                        # TinyStories uses 5e-4, higher values better for small models

    # future you will thank you for descriptive model names
    # TODO: customise this name such that every model you train has a unique identifier!
    config      = model.config 
    model_name  = '-'.join([
        'GPT' if isinstance(model, GPTNeoForCausalLM) else 'BERT',
        f'{model.num_parameters()//1e6:.1f}M',
        f'{config.num_layers if isinstance(model, GPTNeoForCausalLM) else config.num_hidden_layers}L', 
        f'{config.num_heads if isinstance(model, GPTNeoForCausalLM) else config.num_attention_heads}H', 
        f'{config.hidden_size}C',
        f'{config.intermediate_size}I'
    ])

    _train_steps = len(dataset) // (batch_size * gradient_accumulation_steps)
    eval_steps = _train_steps // 10 # evaluate every 10% of training steps

    return dict(
        model_name = model_name,
        batch_size = batch_size, 
        num_train_epochs = num_train_epochs,
        gradient_accumulation_steps = gradient_accumulation_steps,
        lr = lr,
        eval_steps = eval_steps
    )

params_gpt = get_hyperparameters(gpt, train_dataset)
params_rob = get_hyperparameters(rob, train_dataset)

In [52]:
def get_trainer(
        model, tokenizer, train_dataset, eval_dataset, output_dir,
        model_name, batch_size, num_train_epochs, gradient_accumulation_steps, lr, eval_steps):
    ''' more general training arguments you likely want to keep fixed'''

    training_args = TrainingArguments(

        seed       = 42,
        #use_cpu    = False, # use GPU if available (not necessarily faster on laptops, but Apple's MPS have good support)

        output_dir = os.path.join(output_dir, model_name),

        # NOTE: training params
        learning_rate    = lr,
        num_train_epochs = num_train_epochs,
        # Use a smaller batch size to fit into GPU RAM. 
        per_device_train_batch_size = batch_size,
        per_device_eval_batch_size  = batch_size,
        # You should aim to have the same amount of samples per acc step, in all of your experiments!
        # so, if you increase batch_size, decrease gradient_accumulation_steps by the same factor.
        gradient_accumulation_steps = gradient_accumulation_steps,

        # NOTE: Evaluation params
        # wandb is great for tracking experiments, it will even (try to) save your code nowadays
        evaluation_strategy = 'steps',
        eval_steps = eval_steps,
        save_steps = eval_steps,

        logging_first_step=True,
        logging_steps=100,
        report_to  = 'wandb',
    )

    trainer = Trainer(
        model = model, 
        args = training_args, 
        train_dataset = train_dataset, 
        eval_dataset = eval_dataset,
        data_collator = DataCollatorForLanguageModeling(
            tokenizer, mlm=isinstance(model, RobertaForMaskedLM)),
    )

    # print amount of training steps, and how often the model is evaluated
    print(f'''
    Retrieving Trainer for \033[1m{model_name}\033[0m
        training for {num_train_epochs} epochs, {len(train_dataset)} samples
        {batch_size} batch size, {gradient_accumulation_steps} accumulation steps
        gives {len(train_dataset)//(batch_size * gradient_accumulation_steps)} training steps.
        Evaluating every {eval_steps} steps, {len(eval_dataset)} samples 
        ''')

    return trainer

In [53]:
out_dir = './results/models_baseline/' 

trainer_gpt = get_trainer(gpt, tok_gpt, train_dataset, eval_dataset, out_dir, **params_gpt)
trainer_rob = get_trainer(rob, tok_rob, train_dataset, eval_dataset, out_dir, **params_rob)


    Retrieving Trainer for GPT-9.0M-2L-4H-516C-1024I
        training for 1 epochs, 1000 samples
        16 batch size, 16 accumulation steps
        gives 3 training steps.
        Evaluating every 0 steps, 100 samples 
        

    Retrieving Trainer for BERT-9.0M-2L-4H-516C-1024I
        training for 1 epochs, 1000 samples
        16 batch size, 16 accumulation steps
        gives 3 training steps.
        Evaluating every 0 steps, 100 samples 
        


Finally, we can train. 

This configuration takes ≤24hr to pre-train on my M1 Macbook Air with 16GB RAM. Python takes ≤4GB VRAM at a `batch_size=16` and ≤11GB at `batch_size=64`, though they take the same amount of time to train - likely because this processor is not designed to move that much data in and out of RAM constantly. And tbh, the GPU be lacking. If you decide to go the local-training-route, consider [chai](https://github.com/lvillani/chai) to keep your (Apple) laptop awake – there's probably a windows/linux equivalent too. 

In [54]:
def do_train(trainer: Trainer, name: str, out_dir: str): 

    wandb.init(project='tiny-transformers', name=name, group='baseline', config=trainer.args)
    trainer.train()
    trainer.save_model(os.path.join(out_dir, name))

In [55]:
# words vs. tokens 
len(train_dataset['text'][11]), len(train_dataset[11]['input_ids'])

(471, 512)

In [56]:
# do_train(trainer_rob, params_rob['model_name'], out_dir)

In [57]:
# do_train(trainer_gpt, params_gpt['model_name'], out_dir) 

#### Using your Pre-Trained Model 
Try out your own model on some prompts!

In [58]:
# I made a small function for pretty printing into paragraphs with line wrapping for readability
import textwrap 
w = textwrap.TextWrapper(replace_whitespace=False, break_long_words=False, width=60, initial_indent='   ', subsequent_indent='  ')
def see(text): print('\n\033[3m' + '\n\n'.join(['\n'.join(w.wrap(line))
                 for line in text.splitlines() if line.strip() != '']) + '\033[0m\n')

In [59]:
from transformers import AutoTokenizer, AutoModelForCausalLM

# model_name = 'GPT-INFINI-9.0MINFINI-2L-4H-516C-1024I'
model_name = 'GPT-9.0M-2L-4H-516C-1024I'
# model_name = 'BERT-9.0M-2L-4H-516C-1024I' # bert won't work for generation unless you fine-tune it for that task

tokenizer = AutoTokenizer.from_pretrained('10k-tok')
tokenizer.pad_token_id = tokenizer.eos_token_id
model = AutoModelForCausalLM.from_pretrained(f'results/models_baseline/GPT-INFINI-9.0MINFINI-2L-4H-516C-1024I')

#### Inference 
Let's generate a short story like the ones the model has been trained on! You'll notice that the prompt is surrounded by `<s>`, the begin-of-sequence (bos) token, and `</s>` end-of-sequence (eos) / separator (sep) token. This is from the BERT-style tokenisation, making it clear to the model where (one of several) input sequences ends. 

In [60]:
prompt = 'nce upon a time, there was a little car named Beep. Beep loved to go fast and play in the sun. Beep was a healthy car because he always had good fuel. Good fuel made '
input_ids = tokenizer.encode(prompt, return_tensors='pt')

output = model.generate(
    input_ids,                              # input to the model
    max_length=300,                         # maximum generation length
    eos_token_id=tokenizer.eos_token_id,    # early stopping when eos_token is output

    # num_beams=1,                            # number of beams to use in generation
    temperature=1,
)
# print(output)
output_text = tokenizer.decode(output[0])

# textwrap with indentation on every new paragraph
see(output_text)
output


   <s>nce upon a time, there was a little car named Beep.
  Beep loved to go fast and play in the sun. Beep was a
  healthy car because he always had good fuel. Good fuel
  made </s> O O O O O OieMl M M M M M M M M M!" y

   M MM O O O
  Oprprprprprprprprprprprprprprprprprprprprprprprprprprprprprprprprprprprprprprprprprprprprprprprprprprprprprprprprprprprprprprprprprprprprprprprprprprprprprprprprprprprprprprprprprprprprprprprprprprprprprprprprprprprprprprprprprprprprprprprprprprprprprprprprprprprprprprprprprprprprprprprprprprprprprprprprprprprprprprprprprprprprprprprprprprprprprprprprprprprprprprprprprprprprprprprprprprprprprprprprprprprprprprprprprprprprprprprprprprprprprprprprprprpr



tensor([[   1, 1198, 2402,  257,  640,   11,  612,  373,  257, 1310, 1097, 3706,
         1355,  538,   13, 1355,  538, 6151,  284,  467, 3049,  290,  711,  287,
          262, 4252,   13, 1355,  538,  373,  257, 5448, 1097,  780,  339, 1464,
          550,  922, 5252,   13, 4599, 5252,  925,  220,    2,  440,  440,  440,
          440,  440,  440,  494,   44,   75,  337,  337,  337,  337,  337,  337,
          337,  337,  337, 2474,  331,  198,   44,  337,   44,  440,  440,  440,
          440, 1050, 1050, 1050, 1050, 1050, 1050, 1050, 1050, 1050, 1050, 1050,
         1050, 1050, 1050, 1050, 1050, 1050, 1050, 1050, 1050, 1050, 1050, 1050,
         1050, 1050, 1050, 1050, 1050, 1050, 1050, 1050, 1050, 1050, 1050, 1050,
         1050, 1050, 1050, 1050, 1050, 1050, 1050, 1050, 1050, 1050, 1050, 1050,
         1050, 1050, 1050, 1050, 1050, 1050, 1050, 1050, 1050, 1050, 1050, 1050,
         1050, 1050, 1050, 1050, 1050, 1050, 1050, 1050, 1050, 1050, 1050, 1050,
         1050, 1050, 1050, 1

Sounds like me after a few beers too many, but at least the grammar is (mostly) correct. The model also learns some basic reasoning-like associations like being 'so high' allows you to see 'the whole world'. 